In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import urllib.parse
import re
import numpy as np
import seaborn as sns
import ast
from wordcloud import WordCloud, STOPWORDS
import sklearn
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import resample
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import SelectFromModel
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

In [30]:
df = pd.read_parquet('/Users/lijiazheng/Desktop/completion_model_embeddings.parquet')
df

KeyboardInterrupt: 

In [ ]:
df['status_binary'] = (df['statusModule_overallStatus'] == 'COMPLETED').astype(int)
print(df['status_binary'].value_counts())

df = df.drop(columns = ['identificationModule_nctId','statusModule_overallStatus', 'combined_description'])

status_binary
1    223519
0     39929
Name: count, dtype: int64


In [ ]:
from sklearn.metrics import classification_report
# === CONFIG ===
n_runs = 5
desired_ratio = 1
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
        clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
        grid = GridSearchCV(clf, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===

print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))


===================== Run 1/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 2/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 3/5 =====================
Optimal PCA components: 317


...


===================== Run 4/5 =====================
Optimal PCA components: 317


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 5/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Top Features Without Embeddings =====================
designModule_enrollmentInfo_count: 0.2307
healthyVolunteers_yes: 0.1141
healthyVolunteers_no: 0.0758
FdaRegulatedDrug_yes: 0.0677
interventionModel_PARALLEL: 0.0424
Neoplasms: 0.0392
duration_of_trial: 0.0344
FdaRegulatedDevice_yes: 0.0331
organization_class_INDUSTRY: 0.0329
interventionModel_CROSSOVER: 0.0319
facility_count: 0.0314
PHASE3: 0.0262
FdaRegulatedDrug_no: 0.0236
PHASE2: 0.0231
leadsponsor_INDUSTRY: 0.0231
masking_SINGLE: 0.0227
interventionModel_SEQUENTIAL: 0.0224
FdaRegulatedDevice_no: 0.0208
oversightHasDmc_yes: 0.0202
PHASE4: 0.0199

===== Aggregated Classification Report (No Embeddings) =====
  class    precision       recall     f1-score         support
0     0  0.85 ± 0.00  0.73 ± 0.00  0.78 ± 0.00  7986.00 ± 0.00
1     1  0.76 ± 0.00  0.87 ± 0.00  0.81 ± 0.00  7986.00 ± 0.00

===== Aggregated Classification Report (With Embeddings) =====
  class    precision       recall     f1-score       

In [ ]:
results 

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_maximumAge': 0.01825971,
               'designModule_enrollmentInfo_count': 0.21953008,
               'leadsponsor_INDUSTRY': 0.019116016,
               'PHASE2': 0.027924849,
               'PHASE3': 0.026242325,
               'organization_class_INDUSTRY': 0.04184658,
               'oversightHasDmc_no': 0.017507426,
               'oversightHasDmc_yes': 0.023333177,
               'FdaRegulatedDrug_yes': 0.069107555,
               'FdaRegulatedDevice_no': 0.024310622,
               'FdaRegulatedDevice_yes': 0.032334503,
               'designInfo_allocation_NON_RANDOMIZED': 0.019889154,
               'interventionModel_CROSSOVER': 0.037497047,
               'interventionModel_PARALLEL': 0.04372207,
               'primaryPurpose_DIAGNOSTIC': 0.020611009,
               'masking_SINGLE': 0.02604322,
               'healthyVolunteers_no': 0.12593266,
               'healthyVolunteers_yes': 0.0659

In [ ]:
from sklearn.metrics import classification_report
# === CONFIG ===
n_runs = 5
desired_ratio = 1.5
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
        clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
        grid = GridSearchCV(clf, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best parameters for this run: {grid.best_params_}")
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===

print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))


===================== Run 1/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 2/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 3/5 =====================
Optimal PCA components: 318


...


===================== Run 4/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 5/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Top Features Without Embeddings =====================
designModule_enrollmentInfo_count: 0.2738
healthyVolunteers_no: 0.0944
healthyVolunteers_yes: 0.0943
FdaRegulatedDrug_yes: 0.0637
interventionModel_PARALLEL: 0.0496
Neoplasms: 0.0476
duration_of_trial: 0.0371
organization_class_INDUSTRY: 0.0355
facility_count: 0.0336
interventionModel_CROSSOVER: 0.0328
FdaRegulatedDevice_yes: 0.0324
PHASE3: 0.0265
FdaRegulatedDrug_no: 0.0263
PHASE2: 0.0249
FdaRegulatedDevice_no: 0.0249
interventionModel_SINGLE_GROUP: 0.0243
interventionModel_SEQUENTIAL: 0.0234
piSponsorEmployee_yes: 0.0233
leadsponsor_INDUSTRY: 0.0227
primaryPurpose_HEALTH_SERVICES_RESEARCH: 0.0217

===== Aggregated Classification Report (No Embeddings) =====
  class    precision       recall     f1-score          support
0     0  0.86 ± 0.00  0.66 ± 0.01  0.75 ± 0.00   7986.00 ± 0.00
1     1  0.80 ± 0.00  0.93 ± 0.00  0.86 ± 0.00  11979.00 ± 0.00

===== Aggregated Classification Report (With Embeddings) =====

In [ ]:
results

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_maximumAge': 0.01644414,
               'designModule_enrollmentInfo_count': 0.29337928,
               'PHASE2': 0.023827525,
               'PHASE3': 0.02721443,
               'PHASE4': 0.018279687,
               'organization_class_INDUSTRY': 0.031009821,
               'oversightHasDmc_no': 0.01442722,
               'oversightHasDmc_yes': 0.022572191,
               'FdaRegulatedDrug_no': 0.02236405,
               'FdaRegulatedDrug_yes': 0.072984755,
               'FdaRegulatedDevice_no': 0.025595173,
               'interventionModel_CROSSOVER': 0.033931755,
               'interventionModel_PARALLEL': 0.049028024,
               'healthyVolunteers_no': 0.10487092,
               'healthyVolunteers_yes': 0.08691257,
               'duration_of_trial': 0.038068376,
               'facility_count': 0.03279878,
               'Neoplasms': 0.047984026,
               'Respiratory Tract Diseases': 0.

In [ ]:
# === CONFIG ===
n_runs = 5
desired_ratio = 2
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
        clf = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
        grid = GridSearchCV(clf, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best parameters for this run: {grid.best_params_}")
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===

print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))


===================== Run 1/3 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 2/3 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.


===================== Run 3/3 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Top Features Without Embeddings =====================
designModule_enrollmentInfo_count: 0.2865
healthyVolunteers_no: 0.0963
healthyVolunteers_yes: 0.0848
FdaRegulatedDrug_yes: 0.0657
interventionModel_PARALLEL: 0.0522
Neoplasms: 0.0475
FdaRegulatedDevice_yes: 0.0387
duration_of_trial: 0.0350
interventionModel_CROSSOVER: 0.0349
organization_class_INDUSTRY: 0.0334
facility_count: 0.0308
designInfo_allocation_RANDOMIZED: 0.0257
PHASE2: 0.0240
PHASE3: 0.0235
oversightHasDmc_yes: 0.0217
FdaRegulatedDevice_no: 0.0215
leadsponsor_INDUSTRY: 0.0213
FdaRegulatedDrug_no: 0.0201
interventionModel_SEQUENTIAL: 0.0200
primaryPurpose_DIAGNOSTIC: 0.0195

===== Aggregated Classification Report (No Embeddings) =====
  class    precision       recall     f1-score          support
0     0  0.86 ± 0.00  0.62 ± 0.01  0.72 ± 0.00   7986.00 ± 0.00
1     1  0.83 ± 0.00  0.95 ± 0.00  0.89 ± 0.00  15972.00 ± 0.00

===== Aggregated Classification Report (With Embeddings) =====
  class    pr

In [ ]:
results

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_maximumAge': 0.017590467,
               'designModule_enrollmentInfo_count': 0.28743276,
               'PHASE2': 0.024211343,
               'PHASE3': 0.026360098,
               'PHASE4': 0.018560467,
               'organization_class_INDUSTRY': 0.029907247,
               'oversightHasDmc_yes': 0.022006279,
               'FdaRegulatedDrug_yes': 0.06474012,
               'FdaRegulatedDevice_no': 0.020305378,
               'FdaRegulatedDevice_yes': 0.03869102,
               'designInfo_allocation_RANDOMIZED': 0.026385076,
               'interventionModel_CROSSOVER': 0.03780738,
               'interventionModel_PARALLEL': 0.048835423,
               'healthyVolunteers_no': 0.11216473,
               'healthyVolunteers_yes': 0.0741268,
               'duration_of_trial': 0.03783811,
               'facility_count': 0.033590637,
               'Neoplasms': 0.04804359,
               'Respiratory Tra

In [ ]:
n_runs = 5
desired_ratio = 2
# param_grid = {
#     'n_estimators': [300, 400, 500],
#     'max_depth': [4, 6, 8],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'subsample': [0.7, 0.8],
#     'colsample_bytree': [0.7, 0.8]
# }

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names
        print("length of features:", len(selected_feature_names))
        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)


===================== Run 1/5 =====================
Optimal PCA components: 318
length of features: 20


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


length of features: 93


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 2/5 =====================
Optimal PCA components: 318
length of features: 22


...

/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(



===================== Run 5/5 =====================
Optimal PCA components: 318
length of features: 25


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


length of features: 103


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:
print("1:1")
print(np.mean([22,24,29,25,23]), "+/-", np.std([22,24,29,25,23]))
print(np.mean([108, 104, 103, 112, 109]), "+/-", np.std([108, 104, 103, 112, 109]))
print("1:1.5")
print(np.mean([20,21,23,19,21]), "+/-", np.std([20,21,23,19,21]))
print(np.mean([104,97,94,102,107]), "+/-", np.std([104,97,94,102,107]))
print("1:2")
print(np.mean([20,22,24,23,25]), "+/-", np.std([20,22,24,23,25]))
print(np.mean([93,94,104,95,103]), "+/-", np.std([93,94,104,95,103]))

1:1
24.6 +/- 2.4166091947189146
107.2 +/- 3.3105890714493698
1:1.5
20.8 +/- 1.3266499161421599
100.8 +/- 4.707440918375928
1:2
22.8 +/- 1.7204650534085253
97.8 +/- 4.707440918375928


In [ ]:
# Random forest (downsample)

In [27]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# === CONFIG ===
n_runs = 5
desired_ratio = 1
param_grid_rf = {
    'n_estimators': [400, 500, 600],
    'max_depth': [15, 20, 30],
    # 'max_features': ['sqrt', 'log2']
}

# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
        clf = RandomForestClassifier(random_state=42)
        grid = GridSearchCV(clf, param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)

        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best parameters for this run: {grid.best_params_}")
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===

print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))


===================== Run 1/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 500}
Best parameters for this run: {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500}

===================== Run 2/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 400}
Best parameters for this run: {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500}

===================== Run 3/5 =====================
Optimal PCA components: 317


...

Best parameters for this run: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 500}
Best parameters for this run: {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500}

===================== Run 4/5 =====================
Optimal PCA components: 317


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 500}
Best parameters for this run: {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500}

===================== Run 5/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 500}
Best parameters for this run: {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500}

===================== Top Features Without Embeddings =====================
designModule_enrollmentInfo_count: 0.5675
duration_of_trial: 0.1098
facility_count: 0.0519
exclusion_count: 0.0443
eligibilityModule_maximumAge: 0.0432
inclusion_count: 0.0410
eligibilityModule_minimumAge: 0.0329
healthyVolunteers_no: 0.0245
FdaRegulatedDrug_yes: 0.0241
healthyVolunteers_yes: 0.0224
Neoplasms: 0.0171
PHASE2: 0.0144
masking_NONE: 0.0123
FdaRegulatedDevice_no: 0.0115
oversightHasDmc_yes: 0.0114
primaryPurpose_TREATMENT: 0.0099

===== Aggregated Classification Report (No Embeddings) =====
  class    precision       recall     f1-score         support
0     0  0.85 ± 0.01  0.70 ± 0.00  0.76 ± 0.00  7986.00 ± 0.00
1     1  0.74 ± 0.00  0.87 ± 0.01  0.80 ± 0.00  7986.00 ± 0.00

===== Aggregated Classification Report 

In [28]:
results

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_maximumAge': 0.04444847212317863,
               'designModule_enrollmentInfo_count': 0.5811480312303777,
               'PHASE2': 0.014106096300136829,
               'FdaRegulatedDrug_yes': 0.02473933431637326,
               'healthyVolunteers_no': 0.02441994108775901,
               'healthyVolunteers_yes': 0.024710433256953674,
               'duration_of_trial': 0.12193405826683373,
               'inclusion_count': 0.04334425162150246,
               'exclusion_count': 0.04680718311334513,
               'facility_count': 0.055566435536414424,
               'Neoplasms': 0.018775763147125292},
              {'eligibilityModule_minimumAge': 0.032435590424891754,
               'eligibilityModule_maximumAge': 0.04103621745814229,
               'designModule_enrollmentInfo_count': 0.5602789962658029,
               'PHASE2': 0.01416904871790621,
               'oversightHasDmc_yes': 0.011393527610475

In [ ]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# === CONFIG ===
param_grid_rf = {
    'n_estimators': [500, 600, 700],
    'max_depth': [15, 20, 30],
    'max_features': ['sqrt', 'log2']
}


# === Step 1: Prepare Dataset ===
df_sampled = df.copy()

# === Step 2: PCA on cls_embedding ===
embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

pca = PCA()
pca.fit(embeddings_matrix)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Optimal PCA components: {optimal_components}")

pca = PCA(n_components=optimal_components)
reduced_embeddings = pca.fit_transform(embeddings_matrix)
reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
df_pca.index = df_sampled.index

# === Step 3: Feature Processing ===
df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
X = df_sampled.drop(columns=['status_binary'])
y = df_sampled['status_binary']

# Identify feature types
numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
dummy_features = [col for col in X.columns if col not in numerical_features]

# Convert boolean to int
for col in dummy_features:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)

# Create X_emb
X_emb = pd.concat([X, df_pca], axis=1)

# Standardize tabular and embedding features
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])
X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

# === Step 4: Feature Selection ===
def apply_feature_selection(X_train, y_train, X_test):
    selector_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    selector_model.fit(X_train, y_train)
    selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

    selected_mask = selector.get_support()
    selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

    return selector.transform(X_train), selector.transform(X_test), selected_feature_names

X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
    clf = RandomForestClassifier(random_state=42, class_weight='balanced')
    grid = GridSearchCV(clf, param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)

    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    print(f"Best parameters for this run: {grid.best_params_}")
    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # <-- Add classification report here
    report = classification_report(y_test, y_pred, output_dict=True)

    result = {
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'f1': f1,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'report': report  
    }

    if return_importance and feature_names is not None:
        importances = best_model.feature_importances_
        feature_importance_dict = dict(zip(feature_names, importances))
        result['feature_importance'] = feature_importance_dict

    return result

# === Train and Evaluate ===
# Pass selected feature names after feature selection
metrics_no_emb = train_and_eval_all(
    X_train_sel, y_train, X_test_sel, y_test,
    feature_names=selected_feature_names,
    return_importance=True
)

metrics_with_emb = train_and_eval_all(
    X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

# Save metrics to results
for metric, value in metrics_no_emb.items():
    results[f'no_emb_{metric}'].append(value)
for metric, value in metrics_with_emb.items():
    results[f'with_emb_{metric}'].append(value)

results['no_emb_reports'].append(metrics_no_emb['report'])
results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===
print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))

Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 500}
Best parameters for this run: {'max_depth': 30, 'max_features': 'sqrt', 'n_estimators': 500}

===================== Top Features Without Embeddings =====================
designModule_enrollmentInfo_count: 0.6244
duration_of_trial: 0.1260
facility_count: 0.0561
eligibilityModule_maximumAge: 0.0443
inclusion_count: 0.0430
healthyVolunteers_no: 0.0267
FdaRegulatedDrug_yes: 0.0247
healthyVolunteers_yes: 0.0231
Neoplasms: 0.0176
PHASE2: 0.0142

===== Aggregated Classification Report (No Embeddings) =====
  class    precision       recall     f1-score          support
0     0  0.60 ± 0.00  0.64 ± 0.00  0.62 ± 0.00   7986.00 ± 0.00
1     1  0.93 ± 0.00  0.92 ± 0.00  0.93 ± 0.00  44704.00 ± 0.00

===== Aggregated Classification Report (With Embeddings) =====
  class    precision       recall     f1-score          support
0     0  0.88 ± 0.00  0.48 ± 0.00  0.62 ± 0.00   7986.00 ± 0.00
1     1  0.91 ± 0.

In [8]:
results

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_maximumAge': 0.044312408775606986,
               'designModule_enrollmentInfo_count': 0.6243626441527197,
               'PHASE2': 0.014203183683319529,
               'FdaRegulatedDrug_yes': 0.02466501620625897,
               'healthyVolunteers_no': 0.026723906140434548,
               'healthyVolunteers_yes': 0.023057492839890985,
               'duration_of_trial': 0.1259837211513363,
               'inclusion_count': 0.043027971588192195,
               'facility_count': 0.0560538540388548,
               'Neoplasms': 0.017609801423386254}],
             'no_emb_auc': [0.8657674768196706],
             'no_emb_precision': [0.9343924683730509],
             'no_emb_recall': [0.9235862562634216],
             'no_emb_accuracy': [0.8801480356803948],
             'no_emb_f1': [0.9289579372489903],
             'no_emb_tp': [41288],
             'no_emb_tn': [5087],
             'no_emb_fp': [2899],
   

In [31]:
#Random forest (downsample + rebalance)

# === CONFIG ===
n_runs = 5
desired_ratio = 2
param_grid_rf = {
    'n_estimators': [400, 500, 600],
    'max_depth': [15, 20, 30],
    # 'max_features': ['sqrt', 'log2']
}

# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
        clf = RandomForestClassifier(random_state=42, class_weight='balanced')
        grid = GridSearchCV(clf, param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)

        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best parameters for this run: {grid.best_params_}")
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===

print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))


===================== Run 1/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'n_estimators': 600}
Best parameters for this run: {'max_depth': 30, 'n_estimators': 600}

===================== Run 2/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'n_estimators': 600}
Best parameters for this run: {'max_depth': 30, 'n_estimators': 600}

===================== Run 3/5 =====================
Optimal PCA components: 318


...

Best parameters for this run: {'max_depth': 15, 'n_estimators': 600}
Best parameters for this run: {'max_depth': 30, 'n_estimators': 600}

===================== Run 4/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'n_estimators': 600}
Best parameters for this run: {'max_depth': 30, 'n_estimators': 600}

===================== Run 5/5 =====================
Optimal PCA components: 318


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'max_depth': 15, 'n_estimators': 600}
Best parameters for this run: {'max_depth': 30, 'n_estimators': 600}

===================== Top Features Without Embeddings =====================
designModule_enrollmentInfo_count: 0.6074
duration_of_trial: 0.1325
facility_count: 0.0586
inclusion_count: 0.0468
eligibilityModule_maximumAge: 0.0468
healthyVolunteers_no: 0.0267
FdaRegulatedDrug_yes: 0.0242
healthyVolunteers_yes: 0.0228
Neoplasms: 0.0179
PHASE2: 0.0138
oversightHasDmc_yes: 0.0121

===== Aggregated Classification Report (No Embeddings) =====
  class    precision       recall     f1-score          support
0     0  0.77 ± 0.00  0.66 ± 0.00  0.71 ± 0.00   7986.00 ± 0.00
1     1  0.84 ± 0.00  0.90 ± 0.00  0.87 ± 0.00  15972.00 ± 0.00

===== Aggregated Classification Report (With Embeddings) =====
  class    precision       recall     f1-score          support
0     0  0.88 ± 0.00  0.58 ± 0.00  0.70 ± 0.00   7986.00 ± 0.00
1     1  0.82 ± 0.00  0.96 ± 0.00  0.8

In [32]:
results

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_maximumAge': 0.047597994357433226,
               'designModule_enrollmentInfo_count': 0.6061301344437807,
               'PHASE2': 0.013122923113864619,
               'FdaRegulatedDrug_yes': 0.025247168570931216,
               'healthyVolunteers_no': 0.027648622242689703,
               'healthyVolunteers_yes': 0.024547925482909155,
               'duration_of_trial': 0.13303815646778658,
               'inclusion_count': 0.04678112918807913,
               'facility_count': 0.05907143473095601,
               'Neoplasms': 0.01681451140156965},
              {'eligibilityModule_maximumAge': 0.046127021985294024,
               'designModule_enrollmentInfo_count': 0.6118873913392691,
               'PHASE2': 0.013407665228130687,
               'FdaRegulatedDrug_yes': 0.02514066196492969,
               'healthyVolunteers_no': 0.026507564237411085,
               'healthyVolunteers_yes': 0.0220799026403